In [57]:
using Pkg
Pkg.activate("..")

  Activating project at `~/PhD/CMPBP`


In [58]:
using LinearAlgebra, Statistics, Revise
includet("../src/observables.jl")
includet("../src/truncation.jl")
includet("../src/glauber.jl")
includet("../src/cmpbp.jl")

In [118]:
function build_S(A,B; q=size(A, 1), da=size(A,4), db=size(B,4))
    sum(kron(E(z,zz;q), I(q), I(da), I(db)) for z in 1:q, zz in 1:q if z≠zz) + 
        sum(kron(E(z,z;q), E(x,x;q), kron(A[x,x,z,:,:],I(db))+kron(I(da),B[x,x,z,:,:])) for z in 1:q, x in 1:q) +
        sum(kron(E(z,z;q), E(x,y;q), A[x,y,z,:,:], B[x,y,z,:,:]) for z in 1:q, x in 1:q, y in 1:q if y≠x)
end

function eigmax_S(A,B)
    q = size(A, 1)
    da, db = size(A,4), size(B,4)
    size(A,2)==size(A,3)==size(B,1)==size(B,2)==size(B,3)==q || error("Incompatible dimensions")
    (size(A,5)==da && size(B,5)==db) || error("Incompatible dimensions")

    S = build_S(A,B; q,da,db)
    maximum(real, eigen(S).values)
    # eigen(S)
end

fidelity_eig(A,B) = eigmax_S(A,B) - 0.5 * (eigmax_S(A,A) + eigmax_S(B,B))

fidelity_eig (generic function with 1 method)

In [230]:
B = rand(2,2,2,5,5)
A = rand(2,2,2,5,5)
# A = copy(B) .+ 0.1.*rand.()

@show marginals(B) marginals(A) fidelity(A,B; maxiter=10^6) fidelity_eig(A,B);

marginals(B) = ComplexF64[0.4589858273490893 - 7.771959642199712e-19im, 0.5410141726509107 + 7.77195964219972e-19im]
marginals(A) = ComplexF64[0.5299394723673287 - 2.4426847253885214e-18im, 0.4700605276326712 + 2.442684725388523e-18im]
fidelity(A, B; maxiter = 10 ^ 6) = -0.13673027929536552
fidelity_eig(A, B) = -0.13673027929535664


In [267]:
ascent!(A,B; maxiters=[1000], ηs=[1e-5])
@show marginals(B) marginals(A) fidelity_eig(A,B);

marginals(B) = ComplexF64[0.4589858273490893 - 7.771959642199712e-19im, 0.5410141726509107 + 7.77195964219972e-19im]
marginals(A) = ComplexF64[0.4580534930168783 - 1.3081606810005964e-16im, 0.5419465069831217 + 1.308160681000596e-16im]
fidelity_eig(A, B) = -7.690495912626716e-5


In [61]:
(A.-B)./B

2×2×2×5×5 Array{Float64, 5}:
[:, :, 1, 1, 1] =
 -0.860407    1.55854
 -0.580555  174.967

[:, :, 2, 1, 1] =
 -0.1187     0.582736
  0.0487146  4.16771

[:, :, 1, 2, 1] =
  8.82198   0.649381
 -0.134911  3.40928

[:, :, 2, 2, 1] =
 0.512068  -1.26002
 5.50997    0.592003

[:, :, 1, 3, 1] =
 -0.268572  0.295169
 -1.59105   0.358851

[:, :, 2, 3, 1] =
 17.5041     -4.52991
 -0.0738451   1.26768

[:, :, 1, 4, 1] =
 2.79501  0.0364501
 2.02917  1.65762

[:, :, 2, 4, 1] =
 0.563341   -0.399704
 0.0314018  -0.402478

[:, :, 1, 5, 1] =
 -0.31035    0.769661
 -0.960695  -0.752607

[:, :, 2, 5, 1] =
  4.72264    1.17269
 -0.207498  -0.287344

[:, :, 1, 1, 2] =
 1.60751  0.0883257
 4.07955  1.98035

[:, :, 2, 1, 2] =
 -0.896786  0.0781135
 -0.356336  0.411004

[:, :, 1, 2, 2] =
 -0.422411   2.18186
  0.410649  -0.124794

[:, :, 2, 2, 2] =
 -0.550023  0.255823
 -0.959307  0.474217

[:, :, 1, 3, 2] =
  0.322434  -0.105499
 -0.994367   1.4232

[:, :, 2, 3, 2] =
 -0.679934  -0.658325
  6.23335   -0.8